# Wordle Solver

In [ ]:
# import functions
from wordle_helpers import *

In [ ]:
# help(main)

In [ ]:
# Main function that returns a list of possible words
green_letters = 'SY'
green_positions = [1,5,]
yellow_letters = 'L'
yellow_positions = [4,]
bad_letters = 'CRANEOI'

# Main function that returns a list of possible words
words = main(green_letters, green_positions, yellow_letters, yellow_positions, bad_letters)
print(words)

In [ ]:
# Repetitive letters
letters_df = repetitive_letters(wordle_list=words)
print(letters_df.to_string(index=False))

#### Good words:
SALET, CRANE, SHOUT, AUDIO, SLICE, SLICK, SPLIT, STUDY, INCUR, CHOIR, CROWN, CROWD, CRONE, POISE, OLIVE, DUNCE, BINGO, BISON, MIDST, SPOUT

In [ ]:
# Main function that returns a list of possible words
green_letters = ''
green_positions = []
yellow_letters = 'OLIYS'
yellow_positions = []
bad_letters = ''

# Main function that returns a list of possible words
words = main(
    green_letters,
    green_positions,
    yellow_letters,
    yellow_positions,
    bad_letters,
    wordle_list_path="wordle-possible-words",
)
print(words)

In [ ]:
# help(score)

In [ ]:
df = load_data(file_name='wordle_ranking.csv')
df

In [ ]:
# Update score
score_df = score()
score_df

In [ ]:
# Print results
print(score_df.to_string(index=False))

In [ ]:
# help(reset_score)

#### Set custom score

In [ ]:
# help(set_score)

In [ ]:
# # Set custom score
# df = set_score(m_score=107, b_score=41, draw_score=105)
# df

#### Reset score

In [ ]:
## Reset score
# reset_score()

In [62]:
array = list(range(0, 10, 1))
array

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [63]:
def binary_search(array: list, target: int) -> int:
    low, high = 0, len(array)-1
    message = f"Target {target} not in array."
    assert array[low] <= target <= array[high], message
    
    while low <= high:
        mid = (low+high)//2
        if array[mid] == target:
            return mid
        elif array[mid] < target:
            low = mid+1
        else:
            high = mid-1
    return -1

In [64]:
binary_search(array, target=7)

7